# Computer Vision Coursework Submission (INM460)

**Student name, ID and cohort:** Nicolò Finzi (240041165) - PG


# Notebook Setup
In this section you should include all the code cells required to test your coursework submission. Specifically:

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Define Local Path

In the next cell you should assign to the variable `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` the relative path of this folder in your Google Drive.

**IMPORTANT:** you have to make sure that **all the files required to test your functions are loaded using this variable** (as was the case for all lab tutorials). In other words, do not use in the notebook any absolute paths. This will ensure that the markers can run your functions. Also, **do not use** the magic command `%cd` to change directory.



In [2]:
import os

# TODO: Fill in the Google Drive path where you uploaded the CW_folder_PG
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Computer Vision/CW_folder_PG'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Computer Vision/CW_Folder_PG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['Code', 'CW_Dataset', 'Models', 'Personal_Video', 'test_function.ipynb']


### Load packages

In the next cell you should load all the packages required to test your function.

In [3]:
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00


In [4]:
!pip install lz4

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load
import torch
import torch.nn as nn
from torchvision import models
import cv2
from matplotlib import rc
import matplotlib.animation as animation
from mtcnn import MTCNN
from matplotlib import patches
import torchvision.transforms as transforms
import torchvision
from PIL import Image

### Load models

In the next cell you should load your best performing model (this might consist of more than one file). Avoid to load it within `MaskDetection` to avoid having to reload it each time.

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [7]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 3)  # 3 output classes
                                      # (no mask, correct mask, incorrect mask)

model_ft = model_ft.to(device)

CNN = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'Models','BEST_CNN.pth'))

model_ft.load_state_dict(CNN)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


<All keys matched successfully>

# Test MaskDetection

This section should allow to test the `MaskDetection` function. First, add cells with the code needed to load the necessary subroutines to make `MaskDetection` work.

In [8]:
def MaskDetection(path_to_test):
  # The following code section was taken from lab 4 [1]
  cap = cv2.VideoCapture(path_to_test)

  frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  video = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

  fc = 0
  ret = True
  # Reading each frame into video array + converting to RGB
  while fc < frameCount and ret:
      ret, video[fc] = cap.read()
      video[fc] = cv2.cvtColor(video[fc], cv2.COLOR_BGR2RGB)
      fc += 1

  cap.release()
  # Renders animation in HTML format
  rc('animation', html='jshtml', embed_limit=50)

  fig, ax = plt.subplots(figsize=(5, 3)) # Using a small figure size
                                         # to avoid rendering warnings

  # Next we initialize the face detection function
  # This was taken from lab 8 [2]
  mtcnn = MTCNN()

  # Processing 1 frame every 10 and detecting a face accordingly
  sampled_indices = list(range(0, frameCount, 10))
  # Initalise list to store detectors
  faces_per_frame = []

  # Run face detection on every sampled frame
  for idx in sampled_indices:
    faces = mtcnn.detect_faces(video[idx])
    faces_per_frame.append(faces)

  # Setting the model in evaluation mode
  model_ft.eval()

  # Preprocessing the images
  # These values and image tranforms were taken from lab 8 [2]
  data_means = [0.485, 0.456, 0.406]
  data_stds = [0.229, 0.224, 0.225]
  transform = transforms.Compose([transforms.Resize(256),
                      transforms.CenterCrop(224),
                      transforms.ToTensor(),
                      transforms.Normalize(data_means, data_stds)
                      ])

  # Setting up the animation function
  # The following was taken from lab 4 [1]
  def frame(i):
    idx = sampled_indices[i]
    ax.clear()
    ax.axis('off')
    fig.tight_layout()
    ax.imshow(video[idx])

    # Retrieve faces detected for the current frame
    faces = faces_per_frame[i]
    if faces is not None:
      for face in faces:
        # Extracting bounding box coordinates
        x, y, w, h = face["box"]
        # Cropping in order to isolate the face area and feed that in the CNN
        cropped_face = video[idx][y:y+h, x:x+h]
        # Transforming cropped_face from numpy array to PIL Image
        cropped_face = Image.fromarray(cropped_face)

        # Applying transformations
        face_input = transform(cropped_face)
        face_input = face_input.unsqueeze(0) # CNN expects a batch -> [1,C,H,W]
        # Setting it to GPU device
        face_input = face_input.to(device)

        # Predicting the labels
        with torch.no_grad():
          prediction = model_ft(face_input)
          # Finding class with highest score
          predicted_class = prediction.argmax(dim=1).item()

        # Defining labels
        labels = {0:'No Mask', 1:'Correct Mask', 2:'Incorrect Mask'}
        label = labels[predicted_class]

        # Drawing bounding box
        ax.add_patch(patches.Rectangle(xy=(face["box"][0], face["box"][1]),
                                       width=face["box"][2],
                                       height=face["box"][3],
                                       fill=False,
                                       color='r',
                                       linewidth=2
                                       ))
        # Adding label text
        ax.text(x, y-10, label, color='r', fontsize=10, weight='bold')

    return ax


  # Animation - from lab 4 [1]
  anim = animation.FuncAnimation(fig, frame, frames=len(sampled_indices))
  plt.close()
  return anim

Then, make a call to the `MaskDetection` function to see what results it produces.

In [9]:
# Syntax for the next function is the following:
# MaskDetection(path_to_test)

path_to_test = os.path.join(GOOGLE_DRIVE_PATH, 'Personal_Video/mask_video.mp4')
anim = MaskDetection(path_to_test)
anim

Output hidden; open in https://colab.research.google.com to view.

## References

[1] G. Tarroni, *Lab_04,* unpublished lab tutorial, Dept. of Science and Technology, City St. George's, University of London, 2025.

[2] G. Tarroni, *Lab_08_Solved,* unpublished lab tutorial, Dept. of Science and Technology, City St. George's, University of London, 2025.